# Detecção de Raças de Cachorros utilizando Redes Neurais

## Importações

### Importando biblioteca

In [1]:

from sys import platform
import os
import pandas as pd
from PIL import Image
import numpy as np
import random
from sklearn.model_selection import train_test_split

### Importando Dataset

In [2]:
if platform == "linux" or platform == "linux2":
    folder_path = os.getcwd() + "/" + "low-resolution"
elif platform == "win32":
   folder_path = os.getcwd() + "\\" + "low-resolution"


list_images = []

for subfolder in os.listdir(folder_path):
    subfolder_path = os.path.join(folder_path, subfolder)
    
    if os.path.isdir(subfolder_path):
        for file in os.listdir(subfolder_path):
            
            file_path = os.path.join(subfolder_path, file)
            image_about = {}
            breed = subfolder.split("-")[-1]
            image_about["breed"] = breed
            image_about["filename"] = file
            image_about["path_file"] = file_path
            list_images.append(image_about)

df_images = pd.DataFrame.from_records(list_images)

### Pré-processamento

In [3]:
if platform == "linux" or platform == "linux2":
    os_path = "/" 
elif platform == "win32":
   os_path = "\\"

folder_path = os.getcwd() + os_path + "low-resolution"
slot_size = 20

#Retorna uma tupla com os path de cada imagem e o target randomizados
def load_path_of_dataset():
    arquivos = os.listdir(folder_path)
    subpastas = [subpasta for subpasta in os.listdir(folder_path)]
    imagens_path = []
    labels = []
    for subpasta in subpastas:
        subpasta_path = os.path.join(folder_path, subpasta)
        imagens = os.listdir(subpasta_path)
        imagens_path.extend([subpasta + os_path + s for s in imagens])
        label = subpasta.split("-")[2]
        labels.extend([label for s in imagens])
    minha_lista = list(zip(imagens_path, labels))
    random.shuffle(minha_lista)
    return minha_lista

#função responsavel em realizar a divisão em slots
#recebe como parametro a tupla da função load_path_of_dataset, e o numero da interação para carregar o dataframe
#retorna uma tupla menor
def slot_loading(interacao, imagens_amostradas):
    if interacao <= 0:
        raise ValueError('O valor da interação, deve ser maior que zero ! ')
    elif interacao > (interacao * len(imagens_amostradas)):
        raise ValueError('O valor da interação, é muito grande !')
    
    start_index = (interacao - 1) * slot_size
    end_index = interacao * slot_size
    if end_index > len(imagens_amostradas):
        return imagens_amostradas[start_index:]

    return imagens_amostradas[start_index:end_index], len(imagens_amostradas) // slot_size

#Recebe como parametro uma tupla
#retorna um dataset já pré-processado e formatados
def get_images(imagens_amostradas):
    dataset = pd.DataFrame(columns=['Image', 'Label'])
    img_size_height = 300
    img_size_width = 400
    for imagem_nome in imagens_amostradas:
        # extensao = os.path.splitext(imagem_nome)[1].lower()
        imagem_path = os.path.join(folder_path, imagem_nome[0])
        imagem = Image.open(imagem_path)
        imagem = imagem.resize((img_size_width, img_size_height))  # Redimensiona para um tamanho comum
        imagem_array = np.array(imagem) / 255.0  # Normaliza dividindo por 255
        dataset.loc[len(dataset)] = [imagem_array, imagem_nome[1]]

    return dataset;


#### Exemplo de utilização

In [4]:

path_dataset = load_path_of_dataset()
slot_path_dataset, interations = slot_loading(1,path_dataset)
print("Interations: ", interations)
dataset = get_images(slot_path_dataset);
print(dataset)

Interations:  3521


## Separação do Conjunto de teste e treino

In [ ]:
# Sua tupla
path_dataset = load_path_of_dataset()

# Separando o conjunto de teste e treino
X = [item[0] for item in path_dataset]  # Recursos (features) da tupla
y = [item[1] for item in path_dataset]   # Rótulos (labels) da tupla

# Separando 70% para treino e 30% para teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=42)

# Exibindo as dimensões dos conjuntos de treino e teste
print("Dimensões do conjunto de treino:", len(X_treino), len(y_treino))
print("Dimensões do conjunto de teste:", len(X_teste), len(y_teste))

## Criação do modelo

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential()